#**FINE TUNE BERT BASE UNCASED ON IMDB DATASET#**

In [1]:
!pip install transformers scikit-learn --quiet

In [ ]:
!pip install --upgrade datasets --quiet

In [3]:
from datasets import load_dataset
from transformers import EarlyStoppingCallback

In [4]:
dataset = load_dataset("stanfordnlp/imdb")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [ ]:
type(dataset)

datasets.dataset_dict.DatasetDict

In [ ]:
dataset["train"]["text"][0], dataset["train"]["label"][0]

('I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, e

In [ ]:
dataset["train"].features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['neg', 'pos'], id=None)}

In [6]:
train_data = dataset["train"].shuffle(seed=42).select(range(5000))
test_data = dataset["test"].shuffle(seed=42).select(range(2000))

In [7]:
type(train_data)

datasets.arrow_dataset.Dataset

In [ ]:
print(train_data.features)

{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['neg', 'pos'], id=None)}


In [8]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
text = "I want to play soccer."
tokens = tokenizer.encode_plus(text, return_tensors="pt")

In [ ]:
tokens

{'input_ids': tensor([[ 101, 1045, 2215, 2000, 2377, 4715, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
tokenizer.tokenize(text)

['i', 'want', 'to', 'play', 'soccer', '.']

In [ ]:
# prompt: decode the tokens

decoded_text = tokenizer.decode(tokens['input_ids'][0])
decoded_text

'[CLS] i want to play soccer. [SEP]'

In [ ]:
train_data['text'][0]

'There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier\'s plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have "clairvoyance". People like to compare, to judge, to evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it\'s the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all...'

In [9]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

In [ ]:
# I am learning cloud.
# Raj is playing a lot of soccer.

#uppose max_length of my model is 10.

#[CLS],I, am, learning, cloud,.,[SEP],0,0,0  ----> attention masks [1,1,1,1,1,1,1]
#[CLS],Raj is playing a lot of soccer.[SEP]  ---->[1,1,1,1,1,1,1,1,1,1]

In [10]:
train_data = train_data.map(tokenize_function, batched=True)
test_data = test_data.map(tokenize_function, batched=True)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
train_data

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 5000
})

In [11]:
train_data.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_data.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

In [12]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [14]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="binary")
    acc = accuracy_score(labels, predictions)
    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

In [ ]:
# WE ARE DOING FULL LAYER FINETUNING---> WE ARE UPDATING THE WEIGHTS OF ALL THE 12 LAYERS

In [15]:
#Define Trainer
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./bert_sentiment",
    eval_strategy="epoch",
    save_strategy="epoch",
    #learning_rate = 2e-5
    logging_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to = "none"  #to disable WandB integration

)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)] #it means the early stopping mechanism will monitor a specified metric (e.g., validation loss) and stop training early if the metric doesn't improve for 2 consecutive epochs.
)

# 6. Train the Model
trainer.train()

# 7. Evaluate
results = trainer.evaluate()
print("\nFinal Evaluation Results:")
for key, value in results.items():
    print(f"{key.capitalize()}: {value:.4f}")

/tmp/ipython-input-15-2567213258.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
trainer.save_model("/content/bert_sentiment_model_full")
tokenizer.save_pretrained("/content/bert_sentiment_model_full")

In [ ]:
from google.colab import files

# Download the saved model and tokenizer directories
!zip -r /content/bert_sentiment_model_full.zip /content/bert_sentiment_model_full
files.download('/content/bert_sentiment_model_full.zip')


In [16]:
!unzip -o /content/bert_sentiment_model_full_5k_512_imbd.zip -d /content/

# Load the model and tokenizer from the extracted directory
loaded_tokenizer = BertTokenizer.from_pretrained("/content/content/bert_sentiment_model_full")
loaded_model = BertForSequenceClassification.from_pretrained("/content/content/bert_sentiment_model_full")

print("Model and tokenizer loaded successfully from the zip archive.")


Archive:  /content/bert_sentiment_model_full_5k_512_imbd.zip
   creating: /content/content/bert_sentiment_model_full/
  inflating: /content/content/bert_sentiment_model_full/special_tokens_map.json  
  inflating: /content/content/bert_sentiment_model_full/training_args.bin  
  inflating: /content/content/bert_sentiment_model_full/vocab.txt  
  inflating: /content/content/bert_sentiment_model_full/model.safetensors  
  inflating: /content/content/bert_sentiment_model_full/tokenizer_config.json  
  inflating: /content/content/bert_sentiment_model_full/config.json  
Model and tokenizer loaded successfully from the zip archive.


In [17]:
#imdb
#rotten tomatoes

In [18]:
rotten_tomatoes_dataset = load_dataset("rotten_tomatoes", split="test")

README.md:   0%|          | 0.00/7.46k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [19]:
# Take the first 500 reviews
rotten_tomatoes_subset = rotten_tomatoes_dataset.select(range(500))

In [20]:
def tokenize_function_rt(examples):
    return loaded_tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

rotten_tomatoes_subset_tokenized = rotten_tomatoes_subset.map(tokenize_function_rt, batched=True)#1000

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [21]:
rotten_tomatoes_subset_tokenized.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])


In [23]:
rt_eval_trainer = Trainer(
    model=loaded_model,
    args=TrainingArguments(
        output_dir="./rt_evaluation", # Output directory is needed even for evaluation
        per_device_eval_batch_size=8,
        report_to = "none",
    ),
    eval_dataset=rotten_tomatoes_subset_tokenized,
    compute_metrics=compute_metrics, # Use the same compute_metrics function
    tokenizer=loaded_tokenizer,
)

/tmp/ipython-input-23-3273475804.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  rt_eval_trainer = Trainer(


In [24]:
rt_results = rt_eval_trainer.evaluate()

In [26]:
for key, value in rt_results.items():
    print(f"{key.capitalize()}: {value:.4f}")

Eval_loss: 0.5946
Eval_model_preparation_time: 0.0098
Eval_accuracy: 0.8820
Eval_precision: 1.0000
Eval_recall: 0.8820
Eval_f1: 0.9373
Eval_runtime: 13.0991
Eval_samples_per_second: 38.1710
Eval_steps_per_second: 4.8090


In [27]:

import pandas as pd
import torch

# Get predictions for the Rotten Tomatoes subset
predictions_output = rt_eval_trainer.predict(rotten_tomatoes_subset_tokenized)
predictions = predictions_output.predictions
predicted_labels = torch.argmax(torch.tensor(predictions), dim=1).tolist()
actual_labels = rotten_tomatoes_subset["label"]

# Create the DataFrame
eval_df = pd.DataFrame({
    'text': rotten_tomatoes_subset["text"],
    'prediction': predicted_labels,
    'actual': actual_labels
})

print(eval_df.head())


                                                text  prediction  actual
0  lovingly photographed in the manner of a golde...           1       1
1              consistently clever and suspenseful .           1       1
2  it's like a " big chill " reunion of the baade...           0       1
3  the story gives ample opportunity for large-sc...           1       1
4                  red dragon " never cuts corners .           1       1


In [29]:
eval_df.head(10)

,text,prediction,actual
0,lovingly photographed in the manner of a golde...,1,1
1,consistently clever and suspenseful .,1,1
2,"it's like a "" big chill "" reunion of the baade...",0,1
3,the story gives ample opportunity for large-sc...,1,1
4,"red dragon "" never cuts corners .",1,1
5,fresnadillo has something serious to say about...,1,1
6,throws in enough clever and unexpected twists ...,1,1
7,weighty and ponderous but every bit as filling...,1,1
8,a real audience-pleaser that will strike a cho...,1,1
9,generates an enormous feeling of empathy for i...,1,1
